In [1]:
import time
from preprocess import (
    preprocess_monthly_file, 
    preprocessed_filename,
    preprocess_region,
    read_region,
    read_preprocessed,
    preprocess_region_t,
)

In [2]:
t = (2022, 8, 19, "AM")
# region = ('CONUS', [-126.401171875,24.071240929282325,
#                     -61.36210937500001,49.40003415463647])
region = ('WesternUS',[-125.698046875,31.676476158707615,
                       -101.00078125,49.51429477264348])
sat = "SNPP"

## Once per region
Start by preprocessing the region to get rid of static flare sources. Save that new "swiss cheese" shape off into a geojson file for later.


In [3]:
preprocess_region(region)

## Once per input file

Next process each monthly file into half day files

In [5]:
%%time
preprocess_monthly_file(t, sat)

preprocessing monthly file for 2022-8, SNPP
CPU times: user 31.5 s, sys: 1.32 s, total: 32.9 s
Wall time: 32.9 s


## Once per region and t

Do initial filtering and clustering using the preprocessed region and the half day files

In [3]:
region = read_region(region)

In [4]:
%%time
for d in range(1, 32):
    for ampm in ("AM", "PM"):
        preprocess_region_t((t[0], t[1], d, ampm), sat, region=region)

filtering and clustering 2022-8-1 AM, SNPP, WesternUS
filtering and clustering 2022-8-1 PM, SNPP, WesternUS
filtering and clustering 2022-8-2 AM, SNPP, WesternUS
filtering and clustering 2022-8-2 PM, SNPP, WesternUS
filtering and clustering 2022-8-3 AM, SNPP, WesternUS
filtering and clustering 2022-8-3 PM, SNPP, WesternUS
filtering and clustering 2022-8-4 AM, SNPP, WesternUS
filtering and clustering 2022-8-4 PM, SNPP, WesternUS
filtering and clustering 2022-8-5 AM, SNPP, WesternUS
filtering and clustering 2022-8-5 PM, SNPP, WesternUS
filtering and clustering 2022-8-6 AM, SNPP, WesternUS
filtering and clustering 2022-8-6 PM, SNPP, WesternUS
filtering and clustering 2022-8-7 AM, SNPP, WesternUS
filtering and clustering 2022-8-7 PM, SNPP, WesternUS
filtering and clustering 2022-8-8 AM, SNPP, WesternUS
filtering and clustering 2022-8-8 PM, SNPP, WesternUS
filtering and clustering 2022-8-9 AM, SNPP, WesternUS
filtering and clustering 2022-8-9 PM, SNPP, WesternUS
filtering and clustering 202

## Run the rest of the algorithm

Now you have pre-clustered data at the half day for the region

In [11]:
df = read_preprocessed(t, sat, region=region)

## Experiments with hulls

Trying out next steps around hulls:

In [3]:
from FireObj import Cluster, FirePixel
from FireVector import *

In [4]:
%%time
for d in range(1, 32):
    for ampm in ("AM", "PM"):
        df = read_preprocessed((t[0], t[1], d, ampm), sat, region=region)
        for ic, subset in df.groupby("initial_cid"):
            pixel_locs = subset[["x", "y"]].values
            hull = cal_hull(pixel_locs, sensor="viirs")

CPU times: user 2.68 s, sys: 114 ms, total: 2.8 s
Wall time: 2.8 s


In [32]:
%%time
for d in range(1, 32):
    for ampm in ("AM", "PM"):
        df = read_preprocessed((t[0], t[1], d, ampm), sat, region=region)
        for ic in df.initial_cid.unique():
            pixels = [
                FirePixel(
                    row.x,
                    row.y,
                    row.Lon,
                    row.Lat,
                    row.FRP,
                    row.DS,
                    row.DT,
                    row.ampm,
                    row.YYYYMMDD_HHMM,
                    row.Sat,
                    row.origin,
                )
                for row in df.loc[df.initial_cid == ic].itertuples()
            ]
            cluster = Cluster(ic, pixels, t, sensor="viirs")
            hull = cluster.hull

CPU times: user 3.79 s, sys: 36 ms, total: 3.82 s
Wall time: 3.85 s


In [ ]:
# compute initial hulls
hulls = dict()
for ic, subset in df.groupby("initial_cid"):
    pixel_locs = subset[["x", "y"]].values
    hulls[ic] = cal_hull(pixel_locs)

# store hulls
df = pd.merge(df, pd.Series(hulls, name="hull"), how="outer", left_on="initial_cid", right_index=True)